In [ ]:
#pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import re

In [ ]:
#Create spark session
spark = SparkSession \
    .builder \
    .appName("Inverted Index") \
    .getOrCreate()

In [ ]:
spark.version

In [ ]:
#sc

In [ ]:
!ls /data/

In [ ]:
#Create directory to put csv file
!hdfs dfs -mkdir /data

In [ ]:
#Move csv file into hdfs
!hdfs dfs -copyFromLocal -p data/QueryR.csv /data

In [ ]:
#Verify that file is in hdfs
!hdfs dfs -ls /data

In [ ]:
#Convert and import csv file to RDD
qr = sc.textFile('/data/QueryR.csv')

In [ ]:
from pyspark.rdd import RDD
isinstance(qr, RDD)

In [ ]:
#Clean and split tags (Map). Re.sub was used to remove all special characters and spaces
qr_list = qr.map(lambda x: re.sub("[^a-zA-Z0-9]", " ", x).split())
#[^a-zA-Z0-9]
#[>,\"] .split('<') #doesnt remove whitespace 

In [ ]:
qr_list.take(5)

In [ ]:
#FlatMap is used to remove the list aspect (flatten nested list) of our filtered list and assign every word a value of 1
qr_tuple = qr_list.flatMap(lambda qr_list: [(word, 1) for word in qr_list])

In [ ]:
qr_tuple.take(5)

In [ ]:
#Get the sum of how many times a word is presented (Reduce)
qr_counts = qr_tuple.reduceByKey(lambda total, count: total + count)

In [ ]:
qr_counts.take(5)

In [ ]:
#Convert file to textfile
qr_counts.saveAsTextFile('/data/QueryR_wordcount.txt')

In [ ]:
#!hdfs dfs -ls

In [ ]:
#Move file to local from hdfs for export
!hdfs dfs -copyToLocal -p /data /QueryR_wordcount.txt

In [ ]:
#other method for converting csv file
#df = spark.read.load("/data/QueryR.csv",format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
#df.show(10)

In [ ]:
#for x in df:
#    print(x)

In [ ]:
#from pyspark.sql import functions as F
#df.sort(F.desc("tags")).show()